In [ ]:
import numpy as np
import os
import visvis as vv
import matplotlib.pyplot as plt
import pyproj
import pandas as pd
import matplotlib

import geovis_notebook_version

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass

# Load voxel data

In [ ]:
dir_year = '2018'
dir_month = '11'
dir_day = '15'
dir_experiment_num = '01'
sub_dir = '{}_{}_{}/{}'.format(dir_month, dir_day, dir_year, dir_experiment_num)
experiment_str = '{}-{}-{}-{}'.format(dir_month, dir_day, dir_year, dir_experiment_num)
dir_voxels = '/Volumes/david_hd/obsidian/output/experiments/{}/voxels/'.format(sub_dir)
dir_plot_output = '/Users/davidkohn/Desktop'

In [ ]:
fname_voxels_list = [
     dir_voxels + f for f in os.listdir(dir_voxels) 
    if os.path.isfile(os.path.join(dir_voxels, f))
    and 'voxel' in f
]
print(fname_voxels_list)

In [ ]:
# add voxels in
for idx, fn in enumerate(fname_voxels_list):
    print(idx)
    if idx == 0:
        view = geovis_notebook_version.MasonView(fn)
    else:
        view.add_samples(geovis_notebook_version.MasonView(fn))

In [ ]:
# centre point
C_lon = 116.1
C_lat = -24.85

# mason voxels per axis
V_x = 32
V_y = 32
V_z = 32

# we've specified a 10km x 10km area in the experiments
# length is in metres
length_of_square = 15000
depth = 10000

# pointA is the start of the line, pointB is the end of the line
pointA_ne = (409000, 7244000)
pointB_ne = (414000, 7257000)

In [ ]:
# generate coord_array_ne that represents the mason voxelised posterior grid in (northings, eastings)
gda94 = pyproj.Proj(init='epsg:4283')
mgaz50 = pyproj.Proj(init='epsg:28350')

# centre point
C_lonlat = (C_lon, C_lat)
print('C (longitude, latitude): {}'.format(C_lonlat))
C_ne = pyproj.transform(gda94, mgaz50, *C_lonlat)
print('C (northings, eastings): {}'.format(C_ne))

# define the location of the upper left hand corner edge point
E_ne = tuple(np.array(C_ne) + [-1 * 0.5 * length_of_square, -1 * 0.5 * length_of_square])
print('E (northings, eastings): {}'.format(E_ne))

# all the key points
middle = np.array(C_ne)
half_length = 0.5 * length_of_square
lower_left = middle + [-half_length, -half_length]
top_left = middle + [-half_length, half_length]
top_right = middle + [half_length, half_length]
lower_right = middle + [half_length, -half_length]
point_arr = np.array([middle, lower_left, top_left, top_right, lower_right])

# mason voxels per axis
#coord_array_ne = np.zeros((V_x, V_y), dtype=(float, 2))
y_list = np.linspace(lower_left[1], top_left[1], 32)
x_list = np.linspace(lower_left[0], lower_right[0], 32)
grid_x, grid_y = np.meshgrid(x_list, y_list)

ab_arr = np.array([pointA_ne, pointB_ne])

In [ ]:
lower_left

In [ ]:
#plt.scatter(point_arr[:,0], point_arr[:,1], c='red')
fig = plt.figure(figsize=(10,10))
ax = plt.gca()
rect = matplotlib.patches.Rectangle(lower_left, length_of_square, length_of_square,edgecolor='r',facecolor='none')
plt.plot(ab_arr[:,0], ab_arr[:,1], c='blue', marker='o')
ax.add_patch(rect)
ax.set_xlim([400000, 420000])

In [ ]:
f = plt.figure(figsize=(20,20))
tick_fontsize = 20
ax = plt.gca()
plt.scatter(grid_x, grid_y)
plt.plot(ab_arr[:,0], ab_arr[:,1], c='red', marker='o')
_ = ax.set_xticks(x_list)
_ = ax.set_xticklabels(range(len(x_list)))
_ = ax.set_yticks(y_list)
_ = ax.set_yticklabels(range(len(y_list)))
_ = plt.yticks(fontsize = tick_fontsize)
_ = plt.xticks(fontsize = tick_fontsize)
ax.grid()
plt.tight_layout()
fpath = os.path.join(dir_plot_output, 'chord.eps')
#plt.savefig(fpath)

In [ ]:
# calculate slope between pointA and pointB
northings_diff = (pointB_ne[0] - pointA_ne[0])
eastings_diff = (pointB_ne[1] - pointA_ne[1])
m = northings_diff / eastings_diff
# create array all points along the cross section
new_point_array = []
#modifier = 450
modifier = 0
step = int(np.floor(np.diff(x_list)[0])) + modifier
for idx, d in enumerate(range(pointA_ne[1], pointB_ne[1] + step, step)):
    new_e = (pointA_ne[1] + (step * idx) )
    new_n = m * (step * idx) + pointA_ne[0]
    new_point = (new_n, new_e)
    new_point_array.append(new_point)
new_point_array = np.array(new_point_array)

In [ ]:
x_plot_list = []
y_plot_list = []

x_ne_list = []
y_ne_list = []

for coord_ne_i in new_point_array:
    x_i = coord_ne_i[0]
    y_i = coord_ne_i[1]
    x_true = [x_i > x for x in x_list]
    y_true = [y_i > y for y in y_list]
    #print('point', x_i, y_i)
    if np.any(x_true) and (np.any(y_true)):
        x_max = np.max(np.where(x_true))
        y_max = np.max(np.where(y_true))
        """
        print('cell', x_list[x_max], y_list[y_max])
        print('indices', x_max, y_max)
        print()
        """
        x_plot_list.append(x_max)
        y_plot_list.append(y_max)
        x_ne_list.append(x_i)
        y_ne_list.append(y_i)

In [ ]:
df_plot = pd.DataFrame([x_plot_list, y_plot_list]).T.drop_duplicates()
df_plot_ne = pd.DataFrame([x_ne_list, y_ne_list]).T.drop_duplicates()

In [ ]:
df_plot_ne

In [ ]:
occupancy_array = np.zeros(grid_x.shape)
for x,y in df_plot.values:
    occupancy_array[x,y] = 1

In [ ]:
# plot the voxel occupancy map
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 5
plot_height = 5
fontsize = 20
tick_fontsize = 20

x_label = 'Eastings'
y_label = 'Northings'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
ax.set_adjustable('box-forced')
plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = fontsize)
plt.ylabel(y_label, fontsize = fontsize)

f = ax.imshow(occupancy_array.T, cmap=plt.cm.binary, aspect='auto', origin = 'lower')
ax.set_xticks(range(0, 33, 4))
ax.set_yticks(range(0, 33, 4))
#ax.grid(which='major', color='k', linestyle='-', linewidth=1)
plt.tight_layout()
fpath = os.path.join(dir_plot_output, 'cross-section-line.eps')
#plt.savefig(fpath)

In [ ]:
# select a layer to plot
layer_idx = 1
layer_names = ['layer 0', 'layer 1', 'layer 2']
layer_voxels = view.meanlayer(layer_idx)
print('layer_voxels.shape: {}'.format(layer_voxels.shape))

# get z-values for each surface voxel selected
print(layer_voxels.shape)
# x is third dim
# y is second dim
# z if first dim
x_indices = df_plot[0]
y_indices = df_plot[1]
depth_voxels = []
for x, y in zip(x_indices, y_indices):
    z_i = layer_voxels[:, y, x]
    depth_voxels.append(z_i)
depth_voxels = np.array(depth_voxels).T

In [ ]:
plt.contourf(depth_voxels, origin = 'upper')


In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 5
plot_height = 5
fontsize = 20
tick_fontsize = 29

x_label = 'Distance Along Chord (km)'
y_label = 'Depth (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = fontsize)
plt.ylabel(y_label, fontsize = fontsize)


plt.contourf(depth_voxels, origin = 'upper')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=20)
v = np.linspace(0, 1, 5, endpoint=True)
cbar.set_ticks(v)
cbar.set_clim([0, 1])
cbar.set_label('Probability', fontsize=fontsize)

#ax.set_ylim([0.5, 31.5])
ax.set_ylim([0.5, 4.5])
ytl = ['{:.0f}'.format(ytl_i) for ytl_i in ax.get_yticks()]
ax.set_yticks(np.arange(31.5, 0.5, -5))

#ax.invert_yaxis()
#ax.set_xticks(list(range(0, 28, 4)) + [28])
ax.set_yticklabels(ytl)

plt.tight_layout()

fpath = os.path.join(dir_plot_output, 'cross-section-{}.eps'.format(layer_names[layer_idx]))
#plt.savefig(fpath)

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 10
plot_height = 5
fontsize = 20
tick_fontsize = 29

x_label = 'Distance Along Chord (km)'
y_label = 'Depth (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
plt.xticks(fontsize = tick_fontsize)
plt.yticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = tick_fontsize, labelpad=-12)
plt.ylabel(y_label, fontsize = tick_fontsize)

# this is diagonal distance per voxel
chord_distance = np.round(np.sqrt(np.sum([np.power(point1 - point2, 2) for point1, point2 in zip(pointB_ne,pointA_ne)])), 2)
dist_per_vox = chord_distance/30

xticks = [0.5,15,27.5]
ax.set_xticks(xticks)
xticklabels = ['{:.0f}'.format(xt * dist_per_vox/1000) for xt in xticks]
ax.set_xticklabels(xticklabels)

out = plt.contourf(depth_voxels, origin = 'upper')
cbar_ticks = [0, 1.05/2, 1.05]
cbar = plt.colorbar(out, ticks=cbar_ticks)
cbar.ax.tick_params(labelsize=tick_fontsize)
cbar_ticklabels = ['0\\%', '50\\%', '100\\%']
cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_ticklabels)
cbar_label = 'Probability'
ax.tick_params(axis='x', which='major', pad=-1)
ax.text(28.5,32.5,cbar_label, fontsize=tick_fontsize)

# this is vertical/horizontal distance
dist_per_vox = 10000/32

y_max = 31.5
#ax.set_ylim([0.5, y_max])
yt_km = [0, 2000, 4000]
yt_voxels = [yt_i/dist_per_vox for yt_i in yt_km]
yt_voxels_reversed = [y_max - yt_i for yt_i in yt_voxels]
ax.set_yticks(yt_voxels_reversed)
ytl = ['{:.0f}'.format(yt_i / 1000) for yt_i in yt_km]
# 5000 as we're doing 5km deep
target_depth = 5000
ax.set_ylim([y_max - target_depth/dist_per_vox, y_max])

ax.set_yticklabels(ytl)

#plt.tight_layout()


fpath = os.path.join(dir_plot_output, 'cross-section-small-{}.eps'.format(layer_names[layer_idx]))
#plt.savefig(fpath)

In [ ]:
f = plt.figure(figsize=(20,20))
ax = plt.gca()
plt.scatter(grid_x, grid_y)
plt.plot(ab_arr[:,0], ab_arr[:,1], c='red', marker='o')
plt.scatter(df_plot_ne[0], df_plot_ne[1], c='green', marker='*',s=200)
#plt.imshow(df_plot_ne.values)
_ = ax.set_xticks(x_list)
_ = ax.set_xticklabels(range(len(x_list)))
_ = ax.set_yticks(y_list)
_ = ax.set_yticklabels(range(len(y_list)))
_ = plt.yticks(fontsize = tick_fontsize)
_ = plt.xticks(fontsize = tick_fontsize)
ax.grid()
plt.tight_layout()
fpath = os.path.join(dir_plot_output, 'chord-marked.eps')
#plt.savefig(fpath)

In [ ]:
layer_idx = 1
layer_voxels = view.meanlayer(layer_idx)
surface_map = np.flipud(layer_voxels[0,:,:])

try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 10
plot_height = 10
fontsize = 20
tick_fontsize = 29

x_label = 'Eastings (km)'
y_label = 'Northings (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
out = plt.contourf(surface_map, origin = 'upper')

plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = tick_fontsize)
plt.ylabel(y_label, fontsize = tick_fontsize)

dist = 15000
no_voxels = 32
offset = 0.5
dist_per_voxel = dist / (no_voxels)
ticks_voxels = [0.5, 15.5, 31.5]
#ticks_voxels = [(t/dist_per_voxel + offset) for t in ticks_km]
ticks_km = [t * dist_per_voxel for t in ticks_voxels]
#tick_labels = ['{:.1f}'.format(t/1000) for t in ticks_km]
tick_labels = [0, 7.5, 15]

ax.set_xticks(ticks_voxels)
ax.set_yticks(ticks_voxels)
ax.set_xticklabels(tick_labels)
ax.set_yticklabels(tick_labels)

cbar_ticks = [0, 1.05/2, 1.05]
cbar = plt.colorbar(out, ticks=cbar_ticks)
cbar.ax.tick_params(labelsize=tick_fontsize)
cbar_ticklabels = ['0\\%', '50\\%', '100\\%']
cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_ticklabels)
cbar_label = 'Probability'
ax.set_aspect('equal')
ax.text(24,32.5,cbar_label, fontsize=tick_fontsize)
fpath = os.path.join(dir_plot_output, 'surface.eps')
#plt.savefig(fpath)

In [ ]:
fpath = '/Volumes/david_hd/obsidian/output/experiments/11_15_2018/01/output0.npz'
samples = np.load(fpath)
sensors = pd.read_csv('/Volumes/david_hd/obsidian/output/experiments/11_15_2018/01/fieldobsSensors.csv', names=['x', 'y'], comment='#')
actual = pd.read_csv('/Volumes/david_hd/obsidian/output/experiments/11_15_2018/01/fieldobsReadings.csv', comment='#', header=None)
fr = samples['fieldReadings']
valid_bool = (actual.values != -1).flatten()
actual = actual[valid_bool].values
sensors = sensors.loc[valid_bool, :].reset_index(drop=True)
fr = fr[:,valid_bool]

In [ ]:
a = 0
b = 15000
c = 0.5
d = 31.5
def map_point(x):
    y = (x - a) * ((d - c) / (b - a)) + c
    return(y)

In [ ]:
new_x = sensors['x'].apply(map_point)
new_y = sensors['y'].apply(map_point)
new_sensors = pd.DataFrame([new_x, new_y]).T

In [ ]:
plt.scatter(new_sensors['x'], new_sensors['y'])

In [ ]:
n = fr.shape[0]
fr_uncertainty = fr.sum(axis = 0)/n
color_data = np.round(fr_uncertainty,2) * 100

In [ ]:
surface_map = np.flipud(layer_voxels[0,:,:])

try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 10
plot_height = 10
fontsize = 20
tick_fontsize = 29

x_label = 'Eastings (km)'
y_label = 'Northings (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
out = plt.contourf(surface_map, origin = 'upper')

markers = ['o','*']
marker_idx = 0
layer_dict = {
    0: 'Halfway Gneiss',
    1: 'Durlacher Supersuite'
}
for unq in np.unique(actual):
    unq_bool = (actual == unq).flatten()
    plt.scatter(
        new_sensors.loc[unq_bool, 'x'].values, new_sensors.loc[unq_bool,'y'].values,
        c = color_data[unq_bool],
        marker=markers[marker_idx],
        vmin=0,vmax=100,label=layer_dict[unq],edgecolor='black',linewidth=0.1,
        cmap='YlOrRd'
    )
    marker_idx+=1
cbar_ticks = [0, 50, 100]
cbar = plt.colorbar(ticks=cbar_ticks)
cbar.ax.tick_params(labelsize=20)
cbar_ticklabels = ['0\\%', '50\\%', '100\\%']
cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_ticklabels)


#ax.scatter(new_sensors['x'], new_sensors['y'])
ax.set_xlim([0.5,31.5])
ax.set_ylim([0.5,31.5])

plt.yticks(fontsize = 20)
plt.xticks(fontsize = 20)
plt.xlabel(x_label, fontsize = tick_fontsize)
plt.ylabel(y_label, fontsize = tick_fontsize)

dist = 15000
no_voxels = 32
offset = 0.5
dist_per_voxel = dist / (no_voxels)
ticks_voxels = [0.5, 15.5, 31.5]
#ticks_voxels = [(t/dist_per_voxel + offset) for t in ticks_km]
ticks_km = [t * dist_per_voxel for t in ticks_voxels]
#tick_labels = ['{:.1f}'.format(t/1000) for t in ticks_km]
tick_labels = [0, 7.5, 15]

ax.set_xticks(ticks_voxels)
ax.set_yticks(ticks_voxels)
ax.set_xticklabels(tick_labels)
ax.set_yticklabels(tick_labels)

cbar_ticks = [0, 1.05/2, 1.05]
cbar = plt.colorbar(out, ticks=cbar_ticks)
cbar.ax.tick_params(labelsize=20)
cbar_ticklabels = ['0\\%', '50\\%', '100\\%']
cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_ticklabels)
cbar_label = 'Probability'
ax.set_aspect('equal')
# ax.text(10,35.5,cbar_label, fontsize=tick_fontsize) # for (5,5) size plot
ax.text(21.5,37,cbar_label, fontsize=tick_fontsize) # for (10,10) size plot
fpath = os.path.join(dir_plot_output, 'surface-with-fieldobs.eps')
plt.savefig(fpath)

In [ ]:
import matplotlib.cm as cm

In [ ]:
layer_idx = 1
layer_voxels = view.meanlayer(layer_idx)
surface_map = np.flipud(layer_voxels[0,:,:])
upper_val, lower_val = 0.9, 0.1
cond = (surface_map > upper_val) | (surface_map < lower_val)
#surface_map[cond] = np.nan
#surface_map[surface_map > upper_val] = 1
#surface_map[surface_map < lower_val] = 0

try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass
plot_width = 10
plot_height = 10
fontsize = 20
tick_fontsize = 29

x_label = 'Eastings (km)'
y_label = 'Northings (km)'

f = plt.figure(figsize = (plot_width, plot_height))
ax = plt.gca()
ax.patch.set_facecolor('None')
#out = plt.imshow(surface_map, origin = 'upper',  cmap=cm.gray)
out = plt.contourf(surface_map, origin = 'upper')
current_cmap = matplotlib.cm.get_cmap()
current_cmap.set_bad(color='red')

plt.yticks(fontsize = tick_fontsize)
plt.xticks(fontsize = tick_fontsize)
plt.xlabel(x_label, fontsize = tick_fontsize)
plt.ylabel(y_label, fontsize = tick_fontsize)

dist = 15000
no_voxels = 32
offset = 0.5
dist_per_voxel = dist / (no_voxels)
ticks_voxels = [0.5, 15.5, 31.5]
#ticks_voxels = [(t/dist_per_voxel + offset) for t in ticks_km]
ticks_km = [t * dist_per_voxel for t in ticks_voxels]
#tick_labels = ['{:.1f}'.format(t/1000) for t in ticks_km]
tick_labels = [0, 7.5, 15]

ax.set_xticks(ticks_voxels)
ax.set_yticks(ticks_voxels)
ax.set_xticklabels(tick_labels)
ax.set_yticklabels(tick_labels)

cbar_ticks = [0, 1.05/2, 1.05]
cbar = plt.colorbar(out, ticks=cbar_ticks)
cbar.ax.tick_params(labelsize=tick_fontsize)
cbar_ticklabels = ['0\\%', '50\\%', '100\\%']
cbar.set_ticks(cbar_ticks)
cbar.set_ticklabels(cbar_ticklabels)
cbar_label = 'Probability'
ax.set_aspect('equal')
ax.text(24,32.5,cbar_label, fontsize=tick_fontsize)
fpath = os.path.join(dir_plot_output, 'surface.eps')
#plt.savefig(fpath)

In [ ]:
layer_idx = 1
layer_voxels = view.meanlayer(layer_idx)
surface_map = np.flipud(layer_voxels[0,:,:])
f = plt.figure(figsize=(10,5))
ax = plt.gca()
ax.hist(surface_map)
ax.set_xticks(np.linspace(0, 1, 11))

In [ ]:
((surface_map < 0.9) & (surface_map > 0.1)).sum()

In [ ]:
surface_map.shape

In [ ]:
32 * 32